In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

import dask
import dask.dataframe as dd

ROOT = '/home/ian/data/kaggle/optiver_volatility/'
TRAIN_CSV = os.path.join(ROOT, 'train.csv')

In [2]:
df_train_all = pd.read_csv(TRAIN_CSV)

In [3]:
df_train_all.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [4]:
df_train_all.describe()

,stock_id,time_id,target
count,428932.000000,428932.000000,428932.000000
mean,62.437944,16038.972721,0.003880
std,37.126450,9365.103706,0.002936
min,0.000000,5.000000,0.000105
25%,30.000000,7854.000000,0.002026
50%,63.000000,15853.000000,0.003048
75%,96.000000,23994.000000,0.004738
max,126.000000,32767.000000,0.070321


## What can we learn about the continuous target parameter?

# Book train

For a stock and time_id we have a list of `seconds_in_bucket` of 100s of items.

In [5]:
items_in_folder = os.listdir(os.path.join(ROOT, 'book_train.parquet'))
print(f'There are {len(items_in_folder)} items in the folder and they look like {items_in_folder[:5]}')

There are 112 items in the folder and they look like ['stock_id=59', 'stock_id=58', 'stock_id=23', 'stock_id=17', 'stock_id=109']


In [6]:
df_book_train_stock_0 = pd.read_parquet(os.path.join(ROOT, 'book_train.parquet/stock_id=0'))

In [7]:
df_book_train_stock_0.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100


In [8]:
df_book_train_stock_0.describe()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
count,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000
mean,15980.056908,293.692015,0.999495,1.000526,0.999298,1.000727,113.717102,100.825350,86.770241,83.203067
std,9381.778917,173.596440,0.003647,0.003678,0.003660,0.003704,108.657209,109.063892,90.602588,94.968388
min,5.000000,0.000000,0.938241,0.944337,0.937213,0.944456,1.000000,1.000000,1.000000,1.000000
25%,7759.000000,142.000000,0.998368,0.999223,0.998184,0.999390,22.000000,14.000000,18.000000,14.000000
50%,15772.000000,292.000000,0.999632,1.000331,0.999472,1.000496,100.000000,93.000000,100.000000,90.000000
75%,23834.000000,444.000000,1.000753,1.001560,1.000587,1.001745,157.000000,117.000000,102.000000,102.000000
max,32767.000000,599.000000,1.045641,1.056892,1.043756,1.057676,3221.000000,16608.000000,4391.000000,16608.000000


In [9]:
df_book_train_stock_0.query('time_id==5')

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100
...,...,...,...,...,...,...,...,...,...,...
297,5,585,1.003129,1.003749,1.003025,1.003801,100,3,26,3
298,5,586,1.003129,1.003749,1.002612,1.003801,100,3,2,3
299,5,587,1.003129,1.003749,1.003025,1.003801,100,3,26,3
300,5,588,1.003129,1.003749,1.002612,1.003801,100,3,2,3


In [10]:
df_book_train_stock_98 = pd.read_parquet(os.path.join(ROOT, 'book_train.parquet/stock_id=98'))

In [11]:
df_book_train_stock_98.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.000257,1.001263,1.000235,1.001308,2,1,4,1
1,5,1,1.000235,1.001263,1.000212,1.001308,4,1,2,1
2,5,4,1.000235,1.001263,1.000212,1.001308,4,1,2,5
3,5,5,1.000235,1.001263,1.000212,1.001308,4,1,2,12
4,5,6,1.000235,1.001263,1.000212,1.001308,4,1,2,3


In [12]:
df_book_train_stock_98.describe()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
count,905795.000000,905795.000000,905795.000000,905795.000000,905795.000000,905795.000000,905795.000000,905795.000000,905795.000000,905795.000000
mean,16010.265356,293.448323,0.999524,1.000648,0.999314,1.000853,116.728145,85.488035,81.129777,66.753292
std,9363.696022,173.897629,0.003369,0.003404,0.003404,0.003443,124.206204,95.147750,107.703764,96.254298
min,5.000000,0.000000,0.948431,0.955833,0.948245,0.955870,1.000000,1.000000,1.000000,1.000000
25%,7837.000000,142.000000,0.998370,0.999310,0.998175,0.999471,18.000000,7.000000,6.000000,4.000000
50%,15883.000000,291.000000,0.999603,1.000407,0.999442,1.000563,100.000000,90.000000,78.000000,41.000000
75%,23892.000000,444.000000,1.000722,1.001658,1.000559,1.001856,160.000000,104.000000,100.000000,100.000000
max,32767.000000,599.000000,1.046545,1.051670,1.046362,1.055001,9497.000000,4300.000000,9597.000000,6300.000000


In [13]:
df_book_train_stock_98.query('time_id==5')

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.000257,1.001263,1.000235,1.001308,2,1,4,1
1,5,1,1.000235,1.001263,1.000212,1.001308,4,1,2,1
2,5,4,1.000235,1.001263,1.000212,1.001308,4,1,2,5
3,5,5,1.000235,1.001263,1.000212,1.001308,4,1,2,12
4,5,6,1.000235,1.001263,1.000212,1.001308,4,1,2,3
...,...,...,...,...,...,...,...,...,...,...
256,5,590,0.998715,0.999832,0.998580,1.000145,1,6,7,1
257,5,591,0.998715,0.999810,0.998580,0.999832,12,100,7,6
258,5,592,0.998715,0.999273,0.998580,0.999609,11,3,7,1
259,5,593,0.998580,0.999273,0.997820,0.999609,7,4,1,1


# Relationship between taret and bid/ask prices?

In [31]:
#stock_ids = [1, 2, 3]
stock_ids = range(10)

df_train = df_train_all.query('stock_id in @stock_ids')
df_train = df_train.set_index('time_id').drop(columns=['stock_id'])
#df_train.stock_id.value_counts()

In [33]:
df_book_trains = []
for stock_id in stock_ids:
    print(f"Loading stock_id {stock_id}")
    df_book_train_stock_X = pd.read_parquet(os.path.join(ROOT, f'book_train.parquet/stock_id={stock_id}'))
    df_book_trains.append(df_book_train_stock_X)
    
    
df_book_train = pd.concat(df_book_trains)
df_book_train.shape

Loading stock_id 0
Loading stock_id 1
Loading stock_id 2
Loading stock_id 3
Loading stock_id 4
Loading stock_id 5
Loading stock_id 6
Loading stock_id 7
Loading stock_id 8
Loading stock_id 9


(12899265, 10)

In [34]:
df_train.shape, df_book_train.shape

((38300, 1), (12899265, 10))

Observations:

* stock id 0 target vs variance of
  * bid_price1/2  - small positive relationship
  * ask_price1/2 - small positive relationship
  * bid_size1/2 - triangular relationship (ie. pos for small values, negative for larger - maybe?)
  * ask_size1/2 - as for bid size
* stock id 1 - same checks by eye, looks similar

In [35]:
new_cols = ['bid_price1', 'bid_price2', 'ask_price1']
new_series = []
for col in new_cols:
    ser_var_on_field = df_book_train.groupby('time_id')[col].var()
    new_col = col + "_var"
    ser_var_on_field.name = new_col
    new_series.append(ser_var_on_field)

# merged on same time_id
df_features = pd.concat(new_series, axis=1)
df_features.head()

,bid_price1_var,bid_price2_var,ask_price1_var
time_id,,,
5,0.000005,0.000005,0.000006
11,0.000004,0.000004,0.000004
16,0.000003,0.000003,0.000003
31,0.000001,0.000001,0.000001
62,0.000002,0.000002,0.000002


In [36]:
df_train_merged = df_train.merge(df_features, left_index=True, right_index=True)
df_train_merged.head()

,target,bid_price1_var,bid_price2_var,ask_price1_var
time_id,,,,
5,0.004136,0.000005,0.000005,0.000006
5,0.006340,0.000005,0.000005,0.000006
5,0.001848,0.000005,0.000005,0.000006
5,0.005300,0.000005,0.000005,0.000006
5,0.004468,0.000005,0.000005,0.000006


In [37]:
features = df_train_merged.columns.drop('target')
chs = []
for feature in features:
    print(f"Making {feature}")
    chs.append(alt.Chart(df_train_merged.sample(5_000)).mark_point().encode(x=feature, y='target'))
    
# https://altair-viz.github.io/user_guide/compound_charts.html
alt.vconcat(*chs)

Making bid_price1_var
Making bid_price2_var
Making ask_price1_var


alt.VConcatChart(...)

# Simplest ML just to get something working

* make train/test split, stratify on `time_id`
* 3830 rows

**TODO** stratify on time_id between sets

In [38]:
all_rows = df_train_merged.reset_index()

target = 'target'
cols = features
print(f"Features: {cols}")

all_rows.head()

Features: Index(['bid_price1_var', 'bid_price2_var', 'ask_price1_var'], dtype='object')


,time_id,target,bid_price1_var,bid_price2_var,ask_price1_var
0,5,0.004136,0.000005,0.000005,0.000006
1,5,0.006340,0.000005,0.000005,0.000006
2,5,0.001848,0.000005,0.000005,0.000006
3,5,0.005300,0.000005,0.000005,0.000006
4,5,0.004468,0.000005,0.000005,0.000006


In [39]:
X = all_rows[cols]
y = all_rows[target]
stratify_on = all_rows['time_id']

In [1]:
# TODO figure out how to stratify on this!
# TODO CHECK I GOT THIS RIGHT!
unique_time_ids = pd.Series(stratify_on.unique()).sample(frac=1.0).values
#unique_time_ids_half = int(unique_time_ids.shape[0] / 2)
#unique_time_ids, unique_time_ids.shape
# TODO make a mask of 0 or 1 if you're in the first half or the second half

# NO, THIS IS WRONG, I NEED TO MAKE A TRAIN SET WITH ALL OF ONE SET OF TIME IDS AND A TEST SET FOR THE OTHER SET,
# NOT ALLOWING THEM TO MIX BETWEEN. There's no information in the time ids so they can occur in either side. For this 
# trivial ML attempt it doesn't matter, but I need to get it right.

NameError: name 'pd' is not defined

In [41]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [42]:
stratify = stratify_on < np.median(unique_time_ids)
stratify

0         True
1         True
2         True
3         True
4         True
         ...  
38295    False
38296    False
38297    False
38298    False
38299    False
Name: time_id, Length: 38300, dtype: bool

In [43]:
RANDOM_STATE = 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=RANDOM_STATE, stratify=stratify)

In [44]:
est = LinearRegression()
est = RandomForestRegressor()

In [45]:
est.fit(X_train, y_train)

RandomForestRegressor()

In [46]:
est.score(X_test, y_test) # r^2

0.3527103869860003

In [30]:
#est.coef_